# 트랜잭션에 대한 다중 클래스 분류

이 노트북에서는 트랜잭션의 공개 데이터 집합을 미리 정의한 여러 범주로 분류하려고 합니다. 이러한 접근 방식은 트랜잭션 데이터를 미리 정의된 범주에 맞추려는 모든 다중 클래스 분류 사용 사례에 복제할 수 있어야 하며, 이 과정을 마치면 레이블이 지정된 데이터 세트와 레이블이 지정되지 않은 데이터 세트를 모두 처리할 수 있는 몇 가지 접근 방식을 알게 될 것입니다.

이 노트북에서 다룰 다양한 접근 방식은 다음과 같습니다:
- 제로 샷 분류:** 먼저 안내 프롬프트만 사용하여 5개의 명명된 버킷 중 하나에 트랜잭션을 넣는 제로 샷 분류를 해보겠습니다.
- 임베딩을 사용한 분류:** 그 다음에는 레이블이 지정된 데이터 세트에 임베딩을 생성한 다음, 전통적인 분류 모델을 사용해 카테고리를 식별하는 데 있어 임베딩이 얼마나 효과적인지 테스트합니다.
- 미세 조정된 분류:** 마지막으로 라벨이 지정된 데이터 세트에 대해 학습된 미세 조정된 모델을 생성하여 제로 샷 및 소수 샷 분류 접근 방식과 비교합니다.

설정 ## 설정

In [1]:
%load_ext autoreload
%autoreload 
%pip install openai 'openai[datalib]' 'openai[embeddings]' transformers

In [311]:
import openai
import pandas as pd
import numpy as np
import json
import os

openai.api_key = os.getenv("OPENAI_API_KEY")
COMPLETIONS_MODEL = "text-davinci-002"

### 데이터 세트 로드

스코틀랜드 도서관에서 25,000파운드 이상의 공공 거래 데이터 집합을 사용하고 있습니다. 이 데이터 집합에는 세 가지 기능이 있습니다:
- 공급업체: 공급업체의 이름
- 설명: 거래에 대한 텍스트 설명
- Value: GBP로 표시된 거래 금액

**출처**:

https://data.nls.uk/data/organisational-data/transactions-over-25k/

In [312]:
transactions = pd.read_csv('./data/25000_spend_dataset_current.csv', encoding= 'unicode_escape')
len(transactions)

359

In [313]:
transactions.head()

,Date,Supplier,Description,Transaction value (£)
0,21/04/2016,M & J Ballantyne Ltd,George IV Bridge Work,35098.0
1,26/04/2016,Private Sale,Literary & Archival Items,30000.0
2,30/04/2016,City Of Edinburgh Council,Non Domestic Rates,40800.0
3,09/05/2016,Computacenter Uk,Kelvin Hall,72835.0
4,09/05/2016,John Graham Construction Ltd,Causewayside Refurbishment,64361.0


In [335]:
def request_completion(prompt):
    
    completion_response =   openai.Completion.create(
                            prompt=prompt,
                            temperature=0,
                            max_tokens=5,
                            top_p=1,
                            frequency_penalty=0,
                            presence_penalty=0,
                            model=COMPLETIONS_MODEL
                            )
        
    return completion_response

def classify_transaction(transaction,prompt):
    
    prompt = prompt.replace('SUPPLIER_NAME',transaction['Supplier'])
    prompt = prompt.replace('DESCRIPTION_TEXT',transaction['Description'])
    prompt = prompt.replace('TRANSACTION_VALUE',str(transaction['Transaction value (£)']))
    
    classification = request_completion(prompt)['choices'][0]['text'].replace('\n','')
    
    return classification

# This function takes your training and validation outputs from the prepare_data function of the Finetuning API, and
# confirms that each have the same number of classes.
# If they do not have the same number of classes the fine-tune will fail and return an error

def check_finetune_classes(train_file,valid_file):

    train_classes = set()
    valid_classes = set()
    with open(train_file, 'r') as json_file:
        json_list = list(json_file)
        print(len(json_list))

    for json_str in json_list:
        result = json.loads(json_str)
        train_classes.add(result['completion'])
        #print(f"result: {result['completion']}")
        #print(isinstance(result, dict))

    with open(valid_file, 'r') as json_file:
        json_list = list(json_file)
        print(len(json_list))

    for json_str in json_list:
        result = json.loads(json_str)
        valid_classes.add(result['completion'])
        #print(f"result: {result['completion']}")
        #print(isinstance(result, dict))
        
    if len(train_classes) == len(valid_classes):
        print('All good')
        
    else:
        print('Classes do not match, please prepare data again')

제로샷 분류 ## 제로샷 분류

먼저 간단한 프롬프트를 사용하여 이러한 트랜잭션을 분류하는 기본 모델의 성능을 평가해 보겠습니다. 모델에 5개의 카테고리를 제공하고 분류할 수 없는 카테고리에 대해서는 '분류할 수 없음'이라는 포괄적인 항목을 표시합니다.

In [277]:
zero_shot_prompt = '''You are a data expert working for the National Library of Scotland. 
You are analysing all transactions over £25,000 in value and classifying them into one of five categories.
The five categories are Building Improvement, Literature & Archive, Utility Bills, Professional Services and Software/IT.
If you can't tell what it is, say Could not classify
                      
Transaction:
                      
Supplier: SUPPLIER_NAME
Description: DESCRIPTION_TEXT
Value: TRANSACTION_VALUE
                      
The classification is:'''

In [315]:
# Get a test transaction
transaction = transactions.iloc[0]

# Interpolate the values into the prompt
prompt = zero_shot_prompt.replace('SUPPLIER_NAME',transaction['Supplier'])
prompt = prompt.replace('DESCRIPTION_TEXT',transaction['Description'])
prompt = prompt.replace('TRANSACTION_VALUE',str(transaction['Transaction value (£)']))

# Use our completion function to return a prediction
completion_response = request_completion(prompt)
print(completion_response['choices'][0]['text'])

 Building Improvement


첫 번째 시도가 맞습니다. M & J Ballantyne Ltd는 주택 건설업체이며 그들이 수행한 작업은 실제로 건물 개선입니다.

샘플 크기를 25개로 확장하고 간단한 안내 메시지를 통해 다시 한 번 성능을 확인해 보겠습니다.

In [291]:
test_transactions = transactions.iloc[:25]
test_transactions['Classification'] = test_transactions.apply(lambda x: classify_transaction(x,zero_shot_prompt),axis=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [292]:
test_transactions['Classification'].value_counts()

 Building Improvement    14
 Could not classify       5
 Literature & Archive     3
 Software/IT              2
 Utility Bills            1
Name: Classification, dtype: int64

In [293]:
test_transactions.head(25)

,Date,Supplier,Description,Transaction value (£),Classification
0,21/04/2016,M & J Ballantyne Ltd,George IV Bridge Work,35098.0,Building Improvement
1,26/04/2016,Private Sale,Literary & Archival Items,30000.0,Literature & Archive
2,30/04/2016,City Of Edinburgh Council,Non Domestic Rates,40800.0,Utility Bills
3,09/05/2016,Computacenter Uk,Kelvin Hall,72835.0,Software/IT
4,09/05/2016,John Graham Construction Ltd,Causewayside Refurbishment,64361.0,Building Improvement
5,09/05/2016,A McGillivray,Causewayside Refurbishment,53690.0,Building Improvement
6,16/05/2016,John Graham Construction Ltd,Causewayside Refurbishment,365344.0,Building Improvement
7,23/05/2016,Computacenter Uk,Kelvin Hall,26506.0,Software/IT
8,23/05/2016,ECG Facilities Service,Facilities Management Charge,32777.0,Building Improvement
9,23/05/2016,ECG Facilities Service,Facilities Management Charge,32777.0,Building Improvement


레이블이 지정된 예가 없어도 초기 결과는 꽤 괜찮습니다! 분류하지 못한 사례들은 주제에 대한 단서가 거의 없는 어려운 사례들이었지만, 레이블이 지정된 데이터 세트를 정리하여 더 많은 예제를 제공하면 더 나은 성능을 얻을 수 있을 것입니다.

임베딩을 사용한 분류 ##

지금까지 분류한 작은 집합에서 임베딩을 만들어 보겠습니다. 데이터 집합의 101개 트랜잭션에 대해 제로 샷 분류기를 실행하고 얻은 15개의 **분류할 수 없음** 결과를 수동으로 수정하여 레이블이 지정된 예제 집합을 만들었습니다.

### 임베딩 만들기

이 초기 섹션에서는 모든 기능을 연결하는 결합된 필드에서 임베딩을 생성하기 위해 [데이터 세트에서 임베딩 가져오기 노트북](Get_embeddings_from_dataset.ipynb)의 접근 방식을 재사용합니다.

In [317]:
df = pd.read_csv('./data/labelled_transactions.csv')
df.head()

,Date,Supplier,Description,Transaction value (£),Classification
0,15/08/2016,Creative Video Productions Ltd,Kelvin Hall,26866,Other
1,29/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,74806,Building Improvement
2,29/05/2017,Morris & Spottiswood Ltd,George IV Bridge Work,56448,Building Improvement
3,31/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,164691,Building Improvement
4,24/07/2017,John Graham Construction Ltd,Causewayside Refurbishment,27926,Building Improvement


In [318]:
df['combined'] = "Supplier: " + df['Supplier'].str.strip() + "; Description: " + df['Description'].str.strip() + "; Value: " + str(df['Transaction value (£)']).strip()
df.head(2)

,Date,Supplier,Description,Transaction value (£),Classification,combined
0,15/08/2016,Creative Video Productions Ltd,Kelvin Hall,26866,Other,Supplier: Creative Video Productions Ltd; Desc...
1,29/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,74806,Building Improvement,Supplier: John Graham Construction Ltd; Descri...


In [319]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

df['n_tokens'] = df.combined.apply(lambda x: len(tokenizer.encode(x)))
len(df)

101

In [320]:
embedding_path = './data/transactions_with_embeddings_100.csv'

In [321]:
from openai.embeddings_utils import get_embedding

df['babbage_similarity'] = df.combined.apply(lambda x: get_embedding(x, engine='text-similarity-babbage-001'))
df['babbage_search'] = df.combined.apply(lambda x: get_embedding(x, engine='text-search-babbage-doc-001'))
df.to_csv(embedding_path)

### 분류에 임베딩 사용

이제 임베딩을 만들었으니, 임베딩을 우리가 지정한 카테고리로 분류하면 더 많은 성과를 얻을 수 있는지 살펴봅시다.

이를 위해 [Classification_using_embeddings](Classification_using_embeddings.ipynb) 노트북의 템플릿을 사용하겠습니다.

In [309]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from ast import literal_eval

fs_df = pd.read_csv(embedding_path)
fs_df["babbage_similarity"] = fs_df.babbage_similarity.apply(literal_eval).apply(np.array)
fs_df.head()

,Unnamed: 0,Date,Supplier,Description,Transaction value (£),Classification,combined,n_tokens,babbage_similarity,babbage_search
0,0,15/08/2016,Creative Video Productions Ltd,Kelvin Hall,26866,Other,Supplier: Creative Video Productions Ltd; Desc...,136,"[-0.009802100248634815, 0.022551486268639565, ...","[-0.00232666521333158, 0.019198870286345482, 0..."
1,1,29/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,74806,Building Improvement,Supplier: John Graham Construction Ltd; Descri...,140,"[-0.009065819904208183, 0.012094118632376194, ...","[0.005169447045773268, 0.00473341578617692, -0..."
2,2,29/05/2017,Morris & Spottiswood Ltd,George IV Bridge Work,56448,Building Improvement,Supplier: Morris & Spottiswood Ltd; Descriptio...,141,"[-0.009000026620924473, 0.02405017428100109, -...","[0.0028343256562948227, 0.021166473627090454, ..."
3,3,31/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,164691,Building Improvement,Supplier: John Graham Construction Ltd; Descri...,140,"[-0.009065819904208183, 0.012094118632376194, ...","[0.005169447045773268, 0.00473341578617692, -0..."
4,4,24/07/2017,John Graham Construction Ltd,Causewayside Refurbishment,27926,Building Improvement,Supplier: John Graham Construction Ltd; Descri...,140,"[-0.009065819904208183, 0.012094118632376194, ...","[0.005169447045773268, 0.00473341578617692, -0..."


In [310]:
X_train, X_test, y_train, y_test = train_test_split(
    list(fs_df.babbage_similarity.values), fs_df.Classification, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

                      precision    recall  f1-score   support

Building Improvement       0.92      1.00      0.96        11
Literature & Archive       1.00      1.00      1.00         3
               Other       0.00      0.00      0.00         1
         Software/IT       1.00      1.00      1.00         1
       Utility Bills       1.00      1.00      1.00         5

            accuracy                           0.95        21
           macro avg       0.78      0.80      0.79        21
        weighted avg       0.91      0.95      0.93        21



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

이 모델의 성능은 매우 강력하므로 임베딩을 생성하고 더 간단한 분류기를 사용하는 것도 효과적인 접근 방식으로 보이며, 제로 샷 분류기는 라벨이 없는 데이터 세트의 초기 분류를 수행하는 데 도움이 됩니다.

한 걸음 더 나아가, 동일한 레이블이 지정된 데이터 세트에 대해 학습된 미세 조정된 모델이 비슷한 결과를 제공하는지 살펴봅시다.

미세 조정된 거래 분류 ##

이 사용 사례에서는 동일한 레이블이 지정된 101개의 트랜잭션 집합에 대해 미세 조정된 모델을 학습하고 이 미세 조정된 모델을 보이지 않는 트랜잭션 그룹에 적용하여 위의 몇 가지 샷 분류를 개선해 보겠습니다.

미세 조정된 분류기 구축 ###

먼저 데이터를 준비하기 위해 몇 가지 데이터 준비 작업을 수행해야 합니다. 이 작업은 다음 단계를 거치게 됩니다:
- 먼저 클래스를 나열하고 숫자 식별자로 대체합니다. '건물 개선'과 같이 여러 개의 연속된 토큰이 아닌 단일 토큰을 예측하도록 모델을 만들면 더 나은 결과를 얻을 수 있습니다.
- 또한 각 예제에 공통 접두사와 접미사를 추가하여 모델이 예측할 수 있도록 지원해야 합니다. 이 경우 텍스트가 이미 'Supplier'로 시작하므로 접미사 '\n\n###\n\n'을 추가하겠습니다.
- 마지막으로 분류를 위해 각 대상 클래스에 선행 공백을 추가하여 모델을 다시 한 번 지원합니다.

In [210]:
ft_prep_df = fs_df.copy()
len(ft_prep_df)

101

In [211]:
ft_prep_df.head()

,Unnamed: 0,Date,Supplier,Description,Transaction value (£),Classification,combined,n_tokens,babbage_similarity,babbage_search
0,0,15/08/2016,Creative Video Productions Ltd,Kelvin Hall,26866,Other,Supplier: Creative Video Productions Ltd; Desc...,12,"[-0.009630300104618073, 0.009887108579277992, ...","[-0.008217384107410908, 0.025170527398586273, ..."
1,1,29/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,74806,Building Improvement,Supplier: John Graham Construction Ltd; Descri...,16,"[-0.006144719664007425, -0.0018709596479311585...","[-0.007424891460686922, 0.008475713431835175, ..."
2,2,29/05/2017,Morris & Spottiswood Ltd,George IV Bridge Work,56448,Building Improvement,Supplier: Morris & Spottiswood Ltd; Descriptio...,17,"[-0.005225738976150751, 0.015156379900872707, ...","[-0.007611643522977829, 0.030322374776005745, ..."
3,3,31/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,164691,Building Improvement,Supplier: John Graham Construction Ltd; Descri...,16,"[-0.006144719664007425, -0.0018709596479311585...","[-0.007424891460686922, 0.008475713431835175, ..."
4,4,24/07/2017,John Graham Construction Ltd,Causewayside Refurbishment,27926,Building Improvement,Supplier: John Graham Construction Ltd; Descri...,16,"[-0.006144719664007425, -0.0018709596479311585...","[-0.007424891460686922, 0.008475713431835175, ..."


In [212]:
classes = list(set(ft_prep_df['Classification']))
class_df = pd.DataFrame(classes).reset_index()
class_df.columns = ['class_id','class']
class_df  , len(class_df)

(   class_id                 class
 0         0  Literature & Archive
 1         1         Utility Bills
 2         2  Building Improvement
 3         3           Software/IT
 4         4                 Other,
 5)

In [215]:
ft_df_with_class = ft_prep_df.merge(class_df,left_on='Classification',right_on='class',how='inner')

# Adding a leading whitespace onto each completion to help the model
ft_df_with_class['class_id'] = ft_df_with_class.apply(lambda x: ' ' + str(x['class_id']),axis=1)
ft_df_with_class = ft_df_with_class.drop('class', axis=1)

# Adding a common separator onto the end of each prompt so the model knows when a prompt is terminating
ft_df_with_class['prompt'] = ft_df_with_class.apply(lambda x: x['combined'] + '\n\n###\n\n',axis=1)
ft_df_with_class.head()

,Unnamed: 0,Date,Supplier,Description,Transaction value (£),Classification,combined,n_tokens,babbage_similarity,babbage_search,class_id,prompt
0,0,15/08/2016,Creative Video Productions Ltd,Kelvin Hall,26866,Other,Supplier: Creative Video Productions Ltd; Desc...,12,"[-0.009630300104618073, 0.009887108579277992, ...","[-0.008217384107410908, 0.025170527398586273, ...",4,Supplier: Creative Video Productions Ltd; Desc...
1,51,31/03/2017,NLS Foundation,Grant Payment,177500,Other,Supplier: NLS Foundation; Description: Grant P...,11,"[-0.022305507212877274, 0.008543581701815128, ...","[-0.020519884303212166, 0.01993306167423725, -...",4,Supplier: NLS Foundation; Description: Grant P...
2,70,26/06/2017,British Library,Legal Deposit Services,50056,Other,Supplier: British Library; Description: Legal ...,11,"[-0.01019938476383686, 0.015277703292667866, -...","[-0.01843327097594738, 0.03343546763062477, -0...",4,Supplier: British Library; Description: Legal ...
3,71,24/07/2017,ALDL,Legal Deposit Services,27067,Other,Supplier: ALDL; Description: Legal Deposit Ser...,11,"[-0.008471488021314144, 0.004098685923963785, ...","[-0.012966590002179146, 0.01299362163990736, 0...",4,Supplier: ALDL; Description: Legal Deposit Ser...
4,100,24/07/2017,AM Phillip,Vehicle Purchase,26604,Other,Supplier: AM Phillip; Description: Vehicle Pur...,10,"[-0.003459023078903556, 0.004626389592885971, ...","[-0.0010945454705506563, 0.008626140654087067,...",4,Supplier: AM Phillip; Description: Vehicle Pur...


In [236]:
# This step is unnecessary if you have a number of observations in each class
# In our case we don't, so we shuffle the data to give us a better chance of getting equal classes in our train and validation sets
# Our fine-tuned model will error if we have less classes in the validation set, so this is a necessary step

import random 

labels = [x for x in ft_df_with_class['class_id']]
text = [x for x in ft_df_with_class['prompt']]
ft_df = pd.DataFrame(zip(text, labels), columns = ['prompt','class_id']) #[:300]
ft_df.columns = ['prompt','completion']
ft_df['ordering'] = ft_df.apply(lambda x: random.randint(0,len(ft_df)), axis = 1)
ft_df.set_index('ordering',inplace=True)
ft_df_sorted = ft_df.sort_index(ascending=True)
ft_df_sorted.head()

,prompt,completion
ordering,,
0,Supplier: Sothebys; Description: Literary & Ar...,0
1,Supplier: Sotheby'S; Description: Literary & A...,0
2,Supplier: City Of Edinburgh Council; Descripti...,1
2,Supplier: John Graham Construction Ltd; Descri...,2
3,Supplier: John Graham Construction Ltd; Descri...,2


In [ ]:
# This step is to remove any existing files if we've already produced training/validation sets for this classifier
#!rm transactions_grouped*

# We output our shuffled dataframe to a .jsonl file and run the prepare_data function to get us our input files
ft_df_sorted.to_json("transactions_grouped.jsonl", orient='records', lines=True)
!openai tools fine_tunes.prepare_data -f transactions_grouped.jsonl -q

In [322]:
# This functions checks that your classes all appear in both prepared files
# If they don't, the fine-tuned model creation will fail
check_finetune_classes('transactions_grouped_prepared_train.jsonl','transactions_grouped_prepared_valid.jsonl')

31
8
All good


In [ ]:
# This step creates your model
!openai api fine_tunes.create -t "transactions_grouped_prepared_train.jsonl" -v "transactions_grouped_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 5 -m curie

# You can use following command to get fine tuning job status and model name, replace the job name with your job
#!openai api fine_tunes.get -i ft-YBIc01t4hxYBC7I5qhRF3Qdx

In [323]:
# Congrats, you've got a fine-tuned model!
# Copy/paste the name provided into the variable below and we'll take it for a spin
fine_tuned_model = 'curie:ft-personal-2022-10-20-10-42-56'

미세 조정된 분류기 적용하기 ###

이제 분류기를 적용하여 성능을 확인해 보겠습니다. 훈련 집합에는 31개의 고유한 관측값만 있고 검증 집합에는 8개가 있으므로 성능이 어떤지 살펴보겠습니다.

In [324]:
test_set = pd.read_json('transactions_grouped_prepared_valid.jsonl', lines=True)
test_set.head()

,prompt,completion
0,Supplier: Wavetek Ltd; Description: Kelvin Hal...,2
1,Supplier: ECG Facilities Service; Description:...,1
2,Supplier: M & J Ballantyne Ltd; Description: G...,2
3,Supplier: Private Sale; Description: Literary ...,0
4,Supplier: Ex Libris; Description: IT equipment...,3


In [325]:
test_set['predicted_class'] = test_set.apply(lambda x: openai.Completion.create(model=fine_tuned_model, prompt=x['prompt'], max_tokens=1, temperature=0, logprobs=5),axis=1)
test_set['pred'] = test_set.apply(lambda x : x['predicted_class']['choices'][0]['text'],axis=1)

In [326]:
test_set['result'] = test_set.apply(lambda x: str(x['pred']).strip() == str(x['completion']).strip(), axis = 1)

In [327]:
test_set['result'].value_counts()

True     4
False    4
Name: result, dtype: int64

성능은 좋지 않습니다 - 안타깝게도 이는 예상된 결과입니다. 각 클래스에 대한 몇 가지 예제만으로는 임베딩과 기존 분류기를 사용한 위의 접근 방식이 더 효과적이었습니다.

미세 조정된 모델은 레이블이 지정된 관측값이 많을 때 가장 잘 작동합니다. 수백 개 또는 수천 개가 있다면 더 나은 결과를 얻을 수 있지만, 홀드아웃 집합에서 마지막으로 테스트를 수행하여 새로운 관찰 집합에 잘 일반화되지 않는지 확인해 보겠습니다.

In [330]:
holdout_df = transactions.copy().iloc[101:]
holdout_df.head()

,Date,Supplier,Description,Transaction value (£)
101,23/10/2017,City Building LLP,Causewayside Refurbishment,53147.0
102,30/10/2017,ECG Facilities Service,Facilities Management Charge,35758.0
103,30/10/2017,ECG Facilities Service,Facilities Management Charge,35758.0
104,06/11/2017,John Graham Construction Ltd,Causewayside Refurbishment,134208.0
105,06/11/2017,ALDL,Legal Deposit Services,27067.0


In [332]:
holdout_df['combined'] = "Supplier: " + holdout_df['Supplier'].str.strip() + "; Description: " + holdout_df['Description'].str.strip() + '\n\n###\n\n' # + "; Value: " + str(df['Transaction value (£)']).strip()
holdout_df['prediction_result'] = holdout_df.apply(lambda x: openai.Completion.create(model=fine_tuned_model, prompt=x['combined'], max_tokens=1, temperature=0, logprobs=5),axis=1)
holdout_df['pred'] = holdout_df.apply(lambda x : x['prediction_result']['choices'][0]['text'],axis=1)

In [333]:
holdout_df.head(10)

,Date,Supplier,Description,Transaction value (£),combined,prediction_result,pred
101,23/10/2017,City Building LLP,Causewayside Refurbishment,53147.0,Supplier: City Building LLP; Description: Caus...,"{'id': 'cmpl-63YDadbYLo8xKsGY2vReOFCMgTOvG', '...",2
102,30/10/2017,ECG Facilities Service,Facilities Management Charge,35758.0,Supplier: ECG Facilities Service; Description:...,"{'id': 'cmpl-63YDbNK1D7UikDc3xi5ATihg5kQEt', '...",2
103,30/10/2017,ECG Facilities Service,Facilities Management Charge,35758.0,Supplier: ECG Facilities Service; Description:...,"{'id': 'cmpl-63YDbwfiHjkjMWsfTKNt6naeqPzOe', '...",2
104,06/11/2017,John Graham Construction Ltd,Causewayside Refurbishment,134208.0,Supplier: John Graham Construction Ltd; Descri...,"{'id': 'cmpl-63YDbWAndtsRqPTi2ZHZtPodZvOwr', '...",2
105,06/11/2017,ALDL,Legal Deposit Services,27067.0,Supplier: ALDL; Description: Legal Deposit Ser...,"{'id': 'cmpl-63YDbDu7WM3svYWsRAMdDUKtSFDBu', '...",2
106,27/11/2017,Maggs Bros Ltd,Literary & Archival Items,26500.0,Supplier: Maggs Bros Ltd; Description: Literar...,"{'id': 'cmpl-63YDbxNNI8ZH5CJJNxQ0IF9Zf925C', '...",0
107,30/11/2017,Glasgow City Council,Kelvin Hall,42345.0,Supplier: Glasgow City Council; Description: K...,"{'id': 'cmpl-63YDb8R1FWu4bjwM2xE775rouwneV', '...",2
108,11/12/2017,ECG Facilities Service,Facilities Management Charge,35758.0,Supplier: ECG Facilities Service; Description:...,"{'id': 'cmpl-63YDcAPsp37WhbPs9kwfUX0kBk7Hv', '...",2
109,11/12/2017,John Graham Construction Ltd,Causewayside Refurbishment,159275.0,Supplier: John Graham Construction Ltd; Descri...,"{'id': 'cmpl-63YDcML2welrC3wF0nuKgcNmVu1oQ', '...",2
110,08/01/2018,ECG Facilities Service,Facilities Management Charge,35758.0,Supplier: ECG Facilities Service; Description:...,"{'id': 'cmpl-63YDc95SSdOHnIliFB2cjMEEm7Z2u', '...",2


In [334]:
holdout_df['pred'].value_counts()

 2    231
 0     27
Name: pred, dtype: int64

그 결과도 마찬가지로 압도적이었습니다. 따라서 레이블이 지정된 관측값이 적은 데이터 집합에서는 제로 샷 분류 또는 임베딩을 사용한 기존 분류가 미세 조정된 모델보다 더 나은 결과를 반환한다는 사실을 알게 되었습니다.

미세 조정 모델은 여전히 훌륭한 도구이지만, 분류하려는 각 클래스에 대해 레이블이 지정된 예시 수가 많을 때 더 효과적입니다.